# DistilBERT Base Code Training

In [1]:

# Install PyTorch 2.8.0 with CUDA 12.9 support
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu129

# Install HuggingFace Transformers + Datasets for DistilBERT training
%pip install transformers datasets accelerate
%pip install tqdm


Looking in indexes: https://download.pytorch.org/whl/cu129
  Using cached https://download.pytorch.org/whl/cu129/torch-2.9.0%2Bcu129-cp312-cp312-win_amd64.whl.metadata (29 kB)
  Using cached https://download.pytorch.org/whl/cu129/torchvision-0.23.0%2Bcu129-cp312-cp312-win_amd64.whl.metadata (6.3 kB)
  Using cached https://download.pytorch.org/whl/cu129/torchaudio-2.8.0%2Bcu129-cp312-cp312-win_amd64.whl.metadata (7.4 kB)
  Using cached filelock-3.20.0-py3-none-any.whl.metadata (2.1 kB)
  Using cached https://download.pytorch.org/whl/typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached networkx-3.6.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached https://download.pytorch.org/whl/jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.12.0-py3-none-any.whl.metadata (10 kB)
  Using cached https://download.pytorch.org/whl/setuptools-70.2.0-py3-none-any.whl.metadata (5.8 kB)
  Using cach

In [2]:
# Code Cell 1.2 — Simulated 8 GB VRAM Cap (Allocator-Level)

import torch

assert torch.cuda.is_available(), "CUDA is not available."

# Explicit CUDA device index (required)
device_index = 0

# RTX 5060 Ti = 16 GB physical VRAM
TOTAL_VRAM_GB = 16
TARGET_VRAM_GB = 8

memory_fraction = TARGET_VRAM_GB / TOTAL_VRAM_GB

# Limit how much VRAM PyTorch is allowed to reserve
torch.cuda.set_per_process_memory_fraction(memory_fraction, device=device_index)

# Clear any cached allocations and reset stats
torch.cuda.empty_cache()
torch.cuda.reset_peak_memory_stats()

print(
    f"PyTorch VRAM usage capped at ~{TARGET_VRAM_GB} GB "
    f"({memory_fraction:.2f} of total device memory) on cuda:{device_index}"
)


PyTorch VRAM usage capped at ~8 GB (0.50 of total device memory) on cuda:0


In [3]:
# Code Cell 2

import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm import tqdm

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA device:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")


c:\Programming\INDONESIA CONFERENCE PAPER\5060Ti\DistilBERT\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PyTorch version: 2.8.0+cu129
CUDA available: True
CUDA device: NVIDIA GeForce RTX 5060 Ti


In [4]:
from datasets import load_dataset
from transformers import DistilBertTokenizerFast

dataset = load_dataset("ag_news")

print("Dataset splits:", dataset)

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

MAX_LENGTH = 128 

def tokenize_batch(batch):
    return tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=MAX_LENGTH,
    )

tokenized_dataset = dataset.map(tokenize_batch, batched=True)

tokenized_dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "label"]
)

train_dataset = tokenized_dataset["train"]
test_dataset = tokenized_dataset["test"]

print("Sample tokenized batch shape:")
print("Train dataset example input_ids shape:", train_dataset[0]["input_ids"].shape)
print("Train dataset example attention_mask shape:", train_dataset[0]["attention_mask"].shape)
print("Label example:", train_dataset[0]["label"])


Dataset splits: DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})
Sample tokenized batch shape:
Train dataset example input_ids shape: torch.Size([128])
Train dataset example attention_mask shape: torch.Size([128])
Label example: tensor(2)


In [5]:
from torch.utils.data import DataLoader

BATCH_SIZE = 128
NUM_EPOCHS = 30
SUBSET_TRAIN_SIZE = 10_000

print(f"Batch size: {BATCH_SIZE}")
print(f"Planned epochs: {NUM_EPOCHS}")
print(f"Using training subset size: {SUBSET_TRAIN_SIZE}")

train_subset = train_dataset.select(range(SUBSET_TRAIN_SIZE))

NUM_WORKERS = 8

train_loader = DataLoader(
    train_subset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True
)
print("Train batches (subset):", len(train_loader))
print("Test batches:", len(test_loader))
print(f"Dataloaders initialized with num_workers={NUM_WORKERS}, pin_memory=True")

metrics = {
    "epoch": [],
    "train_samples_per_sec": [],
    "epoch_time_sec": [],
    "train_loss": [],
    "val_accuracy": []
}
print("Metric tracking dict initialized:", list(metrics.keys()))


Batch size: 128
Planned epochs: 30
Using training subset size: 10000
Train batches (subset): 79
Test batches: 60
Dataloaders initialized with num_workers=8, pin_memory=True
Metric tracking dict initialized: ['epoch', 'train_samples_per_sec', 'epoch_time_sec', 'train_loss', 'val_accuracy']


In [6]:
# Code Cell 5

from transformers import DistilBertForSequenceClassification

# 1. Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# 2. Load DistilBERT model for 4-class classification (AG News)
num_labels = 4
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=num_labels
)
model.to(device)
print("Model loaded and moved to device.")

# 3. Define optimizer (AdamW is standard for transformers)
learning_rate = 5e-5
optimizer = AdamW(model.parameters(), lr=learning_rate)

print("Optimizer initialized (AdamW) with learning rate:", learning_rate)


Using device: cuda


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded and moved to device.
Optimizer initialized (AdamW) with learning rate: 5e-05


In [7]:
# Code Cell 6

import time
import torch.nn.functional as F
model.train()

def evaluate(model, data_loader, device):
    """Simple evaluation loop to compute validation accuracy."""
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["label"].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )
            logits = outputs.logits
            preds = torch.argmax(logits, dim=-1)

            correct += (preds == labels).sum().item()
            total += labels.size(0)

    model.train() 
    return correct / total if total > 0 else 0.0


total_train_samples = len(train_dataset)

print("Starting training...")
for epoch in range(1, NUM_EPOCHS + 1):
    epoch_start_time = time.time()
    running_loss = 0.0
    num_batches = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch}/{NUM_EPOCHS}"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        num_batches += 1

    epoch_end_time = time.time()
    epoch_time = epoch_end_time - epoch_start_time
    samples_per_sec = total_train_samples / epoch_time if epoch_time > 0 else 0.0
    avg_train_loss = running_loss / num_batches if num_batches > 0 else 0.0

    val_acc = evaluate(model, test_loader, device)

    metrics["epoch"].append(epoch)
    metrics["train_samples_per_sec"].append(samples_per_sec)
    metrics["epoch_time_sec"].append(epoch_time)
    metrics["train_loss"].append(avg_train_loss)
    metrics["val_accuracy"].append(val_acc)

    print(
        f"Epoch {epoch}/{NUM_EPOCHS} | "
        f"Time: {epoch_time:.2f}s | "
        f"Throughput: {samples_per_sec:.2f} samples/s | "
        f"Train Loss: {avg_train_loss:.4f} | "
        f"Val Acc: {val_acc:.4f}"
    )

print("Training complete.")


Starting training...


Epoch 1/30: 100%|██████████| 79/79 [00:35<00:00,  2.22it/s]


Epoch 1/30 | Time: 35.65s | Throughput: 3366.02 samples/s | Train Loss: 0.4826 | Val Acc: 0.8914


Epoch 2/30: 100%|██████████| 79/79 [00:35<00:00,  2.25it/s]


Epoch 2/30 | Time: 35.19s | Throughput: 3410.30 samples/s | Train Loss: 0.2238 | Val Acc: 0.9125


...

Epoch 28/30 | Time: 36.29s | Throughput: 3306.70 samples/s | Train Loss: 0.0072 | Val Acc: 0.9038


Epoch 29/30: 100%|██████████| 79/79 [00:36<00:00,  2.19it/s]


Epoch 29/30 | Time: 36.06s | Throughput: 3328.19 samples/s | Train Loss: 0.0075 | Val Acc: 0.9021


Epoch 30/30: 100%|██████████| 79/79 [00:36<00:00,  2.18it/s]


Epoch 30/30 | Time: 36.30s | Throughput: 3305.69 samples/s | Train Loss: 0.0043 | Val Acc: 0.9036
Training complete.
